## Aggregated Statistics

Here we are looking at the averages metabolite identification accuracy across all datasets. We produce the following tables and figures:
- Table comparing the edge-potential functions (hinge-sigmoid, sigmoid and step-function) (Table S1)
- Pairwise significance tests of the MetFrag 2.2, Bach et al., *Our* and Only MS (Table 4)

In [9]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import itertools as it

from scipy.stats import wilcoxon

sys.path.append(".")
from local_utils import IDIR_CASMI, IDIR_EA

from msmsrt_scorer.experiments.EA_Massbank.plot_and_table_utils import IDIR_METFRAG, _label_p, load_results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
param_selection_measure = "topk_auc"
eval_method = "casmi"
k_values_to_consider = [1, 5, 10, 20]

### Edge-potential Functions

**Load the results:**

In [4]:
MAKE_ORDER_PROB = ["sigmoid", "hinge_sigmoid", "stepfun"]

res = pd.DataFrame()

for make_order_prob, base_dir in zip(MAKE_ORDER_PROB, ["../EA_Massbank/results__TFG__platt", 
                                                       "../EA_Massbank/results__TFG__gridsearch",
                                                       "../EA_Massbank/results__TFG__gridsearch"]):
    # EA Dataset
    for ion_mode, max_n_ms2, n_samples in [("positive", 100, 100), ("negative", 65, 50)]:
        _idir = IDIR_EA(
            tree_method="random", n_random_trees=32, ion_mode=ion_mode, D_value_method=None, mode="application",
            base_dir=base_dir, param_selection_measure=param_selection_measure, 
            make_order_prob=make_order_prob, norm_scores="none", margin_type="max")

        _res = load_results(
            _idir, "MS + RT (our, %s)" % make_order_prob, max_n_ms2, n_samples=n_samples, method=eval_method)[0]
        _res["Dataset"] = "EA (Massbank)"
        _res["Ionization"] = ion_mode
        _res["Function"] = make_order_prob
    
        res = pd.concat((res, _res))

for make_order_prob, base_dir in zip(MAKE_ORDER_PROB, ["../CASMI_2016/results__TFG__platt", 
                                                       "../CASMI_2016/results__TFG__gridsearch",
                                                       "../CASMI_2016/results__TFG__gridsearch"]):
    # CASMI Dataset
    for ion_mode, max_n_ms2, n_samples in [("positive", 75, 50), ("negative", 50, 50)]:
        _idir = IDIR_CASMI(
        tree_method="random", n_random_trees=32, ion_mode=ion_mode, D_value_method=None, mode="application",
        base_dir=base_dir, param_selection_measure=param_selection_measure, 
        make_order_prob=make_order_prob, norm_order_scores=False, margin_type="max")

        _res = load_results(
            _idir, "MS + RT (our, %s)" % make_order_prob, max_n_ms2, n_samples=n_samples, method=eval_method)[0]
        _res["Dataset"] = "CASMI 2016"
        _res["Ionization"] = ion_mode
        _res["Function"] = make_order_prob
        
        res = pd.concat((res, _res))

res = res.drop_duplicates(subset=res.columns[:-1]).drop("sample", axis=1)

assert (res[res.Method == "Only MS"].shape[0] == 250)
assert (res[res.Method != "Only MS"].shape[0] == 750)

**Table:**

In [5]:
res \
    .groupby(["Method", "Dataset", "Ionization", "Function"]).mean() \
    .groupby(["Method", "Function"]).mean() \
    .round(1) \
    .reset_index().drop(["Function", "Top-3"], axis=1)

,Method,Top-1,Top-5,Top-10,Top-20
0,"MS + RT (our, hinge_sigmoid)",21.3,52.5,64.2,74.2
1,"MS + RT (our, sigmoid)",21.2,52.6,63.9,73.9
2,"MS + RT (our, stepfun)",20.3,52.2,63.8,74.2
3,Only MS,16.7,49.5,60.4,70.6


There seems to be not much difference between the different function.

### Pairwise Method Significance tests

In [6]:
res_methods = pd.DataFrame()

# EA Dataset (our, Sigmoid)
for ion_mode, max_n_ms2, n_samples in [("positive", 100, 100), ("negative", 65, 50)]:
    _idir = IDIR_EA(
        tree_method="random", n_random_trees=32, ion_mode=ion_mode, D_value_method=None, mode="application",
        base_dir="../EA_Massbank/results__TFG__platt", param_selection_measure=param_selection_measure, 
        make_order_prob="sigmoid", norm_scores="none", margin_type="max")

    _res = load_results(
        _idir, "MS + RT (our, sigmoid)", max_n_ms2, n_samples=n_samples, method=eval_method)[0]
    _res["Dataset"] = "EA (Massbank)"
    _res["Ionization"] = ion_mode
    _res["Function"] = "sigmoid"

    res_methods = pd.concat((res_methods, _res))
    
# EA Dataset (our, Chain-graph)
for ion_mode, max_n_ms2, n_samples in [("positive", 100, 100), ("negative", 65, 50)]:
    _idir = IDIR_EA(
        tree_method="chain", n_random_trees=None, ion_mode=ion_mode, D_value_method=None, mode="application",
        base_dir="../EA_Massbank/results__TFG__gridsearch", param_selection_measure=param_selection_measure, 
        make_order_prob="hinge_sigmoid", norm_scores="none", margin_type="max")

    _res = load_results(
        _idir, "MS + RT (Chain-graph)", max_n_ms2, n_samples=n_samples, method=eval_method)[0]
    _res["Dataset"] = "EA (Massbank)"
    _res["Ionization"] = ion_mode
    _res["Function"] = "hinge_sigmoid"

    res_methods = pd.concat((res_methods, _res))
    
# EA Dataset (our, MetFrag 2.2)
for ion_mode, max_n_ms2, n_samples in [("positive", 100, 100), ("negative", 65, 50)]:
    _idir = IDIR_METFRAG(
        ion_mode=ion_mode, mode="application", base_dir="../EA_Massbank/results__MetFrag22",
        param_selection_measure=param_selection_measure, pref_model="cdk")

    _res = load_results(
        _idir, "MS + RT (MetFrag 2.2)", max_n_ms2, n_samples=n_samples, method=eval_method)[0]
    _res["Dataset"] = "EA (Massbank)"
    _res["Ionization"] = ion_mode
    _res["Function"] = "none"

    res_methods = pd.concat((res_methods, _res))
    
# CASMI Dataset (our, Sigmoid)
for ion_mode, max_n_ms2, n_samples in [("positive", 75, 50), ("negative", 50, 50)]:
    _idir = IDIR_CASMI(
    tree_method="random", n_random_trees=32, ion_mode=ion_mode, D_value_method=None, mode="application",
    base_dir="../CASMI_2016/results__TFG__platt", param_selection_measure=param_selection_measure, 
    make_order_prob="sigmoid", norm_order_scores=False, margin_type="max")

    _res = load_results(
        _idir, "MS + RT (our, sigmoid)", max_n_ms2, n_samples=n_samples, method=eval_method)[0]
    _res["Dataset"] = "CASMI 2016"
    _res["Ionization"] = ion_mode
    _res["Function"] = "sigmoid"

    res_methods = pd.concat((res_methods, _res))
    
# CASMI Dataset (our, Chain-graph)
for ion_mode, max_n_ms2, n_samples in [("positive", 75, 50), ("negative", 50, 50)]:
    _idir = IDIR_CASMI(
    tree_method="chain", n_random_trees=None, ion_mode=ion_mode, D_value_method=None, mode="application",
    base_dir="../CASMI_2016/results__TFG__gridsearch", param_selection_measure=param_selection_measure, 
    make_order_prob="hinge_sigmoid", norm_order_scores=False, margin_type="max")

    _res = load_results(
        _idir, "MS + RT (Chain-graph)", max_n_ms2, n_samples=n_samples, method=eval_method)[0]
    _res["Dataset"] = "CASMI 2016"
    _res["Ionization"] = ion_mode
    _res["Function"] = "hinge_sigmoid"

    res_methods = pd.concat((res_methods, _res))

        
# CASMI Dataset (our, MetFrag 2.2)
for ion_mode, max_n_ms2, n_samples in [("positive", 75, 50), ("negative", 50, 50)]:
    _idir = IDIR_METFRAG(
        ion_mode=ion_mode, mode="application", base_dir="../CASMI_2016/results__MetFrag22",
        param_selection_measure=param_selection_measure, pref_model="c6d6f521")

    _res = load_results(
        _idir, "MS + RT (MetFrag 2.2)", max_n_ms2, n_samples=n_samples, method=eval_method)[0]
    _res["Dataset"] = "CASMI 2016"
    _res["Ionization"] = ion_mode
    _res["Function"] = "none"

    res_methods = pd.concat((res_methods, _res))

    
res_methods.head()

res_methods = res_methods.drop_duplicates(subset=res_methods.columns[:-1]).drop("sample", axis=1)

assert (res_methods[res_methods.Method == "Only MS"].shape[0] == 250)
assert (res_methods[res_methods.Method != "Only MS"].shape[0] == 750)

In [7]:
res_methods \
    .groupby(["Method", "Dataset", "Ionization", "Function"]).mean() \
    .groupby(["Method", "Function"]).mean() \
    .round(1) \
    .reset_index().drop(["Function", "Top-3"], axis=1)

,Method,Top-1,Top-5,Top-10,Top-20
0,MS + RT (Chain-graph),19.0,51.2,63.4,72.8
1,MS + RT (MetFrag 2.2),20.5,49.1,61.2,72.6
2,"MS + RT (our, sigmoid)",21.2,52.6,63.9,73.9
3,Only MS,16.7,49.5,60.4,70.6


In [10]:
res_sign = pd.DataFrame()

for k in [1, 5, 10, 20]:
    for m1, m2 in it.permutations(["MS + RT (our, sigmoid)", "MS + RT (Chain-graph)", "MS + RT (MetFrag 2.2)"], 2):
        _res_m1 = res_methods.loc[res_methods.Method == m1, "Top-%d" % k].values
        _res_m2 = res_methods.loc[res_methods.Method == m2, "Top-%d" % k].values
    
        _, _p = wilcoxon(_res_m1, _res_m2, alternative="greater")  # H0: ~x = ~y, Alternative x - y > 0 
    
        res_sign = pd.concat((res_sign, pd.DataFrame({"Method-1": [m1], "Method-2": [m2], "p_value": [_p], "k": [k]})))

In [11]:
print("Top - 1")
res_sign[res_sign.k==1].drop("k", axis=1).pivot(index="Method-1", columns="Method-2")

Top - 1


p_value                        \
Method-2               MS + RT (Chain-graph) MS + RT (MetFrag 2.2)   
Method-1                                                             
MS + RT (Chain-graph)                    NaN          1.000000e+00   
MS + RT (MetFrag 2.2)           4.285457e-08                   NaN   
MS + RT (our, sigmoid)          1.726729e-23          1.003832e-07   

                                               
Method-2               MS + RT (our, sigmoid)  
Method-1                                       
MS + RT (Chain-graph)                     1.0  
MS + RT (MetFrag 2.2)                     1.0  
MS + RT (our, sigmoid)                    NaN

In [12]:
print("Top - 5")
res_sign[res_sign.k==5].drop("k", axis=1).pivot(index="Method-1", columns="Method-2")

Top - 5


p_value                        \
Method-2               MS + RT (Chain-graph) MS + RT (MetFrag 2.2)   
Method-1                                                             
MS + RT (Chain-graph)                    NaN          2.499683e-08   
MS + RT (MetFrag 2.2)           1.000000e+00                   NaN   
MS + RT (our, sigmoid)          3.645943e-10          1.289703e-30   

                                               
Method-2               MS + RT (our, sigmoid)  
Method-1                                       
MS + RT (Chain-graph)                     1.0  
MS + RT (MetFrag 2.2)                     1.0  
MS + RT (our, sigmoid)                    NaN

In [13]:
print("Top - 10")
res_sign[res_sign.k==10].drop("k", axis=1).pivot(index="Method-1", columns="Method-2")

Top - 10


p_value                        \
Method-2               MS + RT (Chain-graph) MS + RT (MetFrag 2.2)   
Method-1                                                             
MS + RT (Chain-graph)                    NaN          2.318943e-11   
MS + RT (MetFrag 2.2)               1.000000                   NaN   
MS + RT (our, sigmoid)              0.000007          2.322066e-22   

                                               
Method-2               MS + RT (our, sigmoid)  
Method-1                                       
MS + RT (Chain-graph)                0.999993  
MS + RT (MetFrag 2.2)                1.000000  
MS + RT (our, sigmoid)                    NaN

In [14]:
print("Top - 20")
res_sign[res_sign.k==20].drop("k", axis=1).pivot(index="Method-1", columns="Method-2")

Top - 20


p_value                        \
Method-2               MS + RT (Chain-graph) MS + RT (MetFrag 2.2)   
Method-1                                                             
MS + RT (Chain-graph)                    NaN          8.127166e-04   
MS + RT (MetFrag 2.2)           9.991873e-01                   NaN   
MS + RT (our, sigmoid)          6.333508e-09          3.777201e-11   

                                               
Method-2               MS + RT (our, sigmoid)  
Method-1                                       
MS + RT (Chain-graph)                     1.0  
MS + RT (MetFrag 2.2)                     1.0  
MS + RT (our, sigmoid)                    NaN